# Classification des données en utilisant le SVM

## Importation des données

In [1]:
import pymongo
import sklearn as sk
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
user_collection = db["users_labeled"]

In [ ]:
users = list(user_collection.find({}))
users = pd.DataFrame(users)
users.index = users.user_id

#Normalement ce bloc va être supprimé une fois que le traitement aura été fait ailleurs
users = users.drop(columns=["_id","user_id","last_tweet_published_id","tweet_ids","friends_count","followers_count", "suspicious_fields", "suspicious_score"])
print(users.columns)

## Séparation des données labélisées en Apprentisage , Test et Validation

In [ ]:
Y=users.label
X=users.drop(columns=["label"])

X_tmp, X_test, Y_tmp, Y_test = train_test_split(X, Y, test_size=0.3)
X_app, X_val, Y_app, Y_val = train_test_split(X_tmp, Y_tmp, test_size=0.5)


print(X_app.shape)
print(X_val)
print(X_test.shape)

## Classification avec SVM

### Choix du meilleur Kernel et de la meilleure valeur de C

In [ ]:
kernels=["linear","poly", "rbf","sigmoid"]
C_values=[0.01 ,0.1 ,1 ,10 ,100]
val=[]

for k in kernels:
    for c in C_values:
        svm=SVC(C=c,kernel =k)
        svm.fit(X_app, Y_app)
        Y_pred=svm.predict(X_val)

        erreur=1-metrics.accuracy_score(Y_val,Y_pred)

        print('Pour kernel=',k,' C=',c,', erreur=  ',erreur)
        val.append({"kernel" :k,"C":c,"erreur":erreur})


val.sort(key=lambda x: x['erreur'])

print(val[0])
kernel_opti=val[0]['kernel']
C_opti=val[0]['C']


### Nouvelle itération avec les paramètres optimaux

In [ ]:
svm=SVC(C=C_opti,kernel =kernel_opti)
print(svm)
svm.fit(X_app, Y_app)
y_pred=svm.predict(X_test)
#print(y_val)
#print(y_pred)

erreur=1-metrics.accuracy_score(Y_test,y_pred)

print(erreur)

mat=metrics.confusion_matrix(Y_test,y_pred)
print(mat)

### Recherche des meilleurs attributs au vu de leur impact sur l'erreur

In [ ]:
attributs=users.columns
impact=[]
err_fix=1
verif=True
users_best=users

while (verif):
    for att in attributs :
        users_red=users_best.drop(columns=[att])
        Y=users_red.label
        X=users_red.drop(columns=["label"])

        X_tmp, X_test, Y_tmp, Y_test = train_test_split(X, Y, test_size=0.3)
        X_app, X_val, Y_app, Y_val = train_test_split(X_tmp, Y_tmp, test_size=0.5)
        svm=SVC(C=C_opti,kernel =kernel_opti)
        print(svm)
        svm.fit(X_app, Y_app)
        y_pred=svm.predict(X_test)
        #print(y_val)
        #print(y_pred)

        erreur=1-metrics.accuracy_score(Y_test,y_pred)

        print(erreur)
        impact.append({"attribut" :att,"erreur":erreur})

        mat=metrics.confusion_matrix(Y_test,y_pred)
        print(mat)
    #On retire l'attribut qui contribue le plus à  l'erreur 
    impact.sort(key=lambda x: x['erreur'])
    print(impact[0])
    attribut_neg=val[0]['attribut']
    err_cal=val[0]['erreur']
    users_best=users.drop(columns=[attribut_neg])

    attributs.remove(attribut_neg)
    if(err_cal>err_fix):
        verif=False
    else:
        err_fix=err_cal
    if(len(attributs)==2):
        verif=False


### ACP 

## Représentation graphique